## Setting up

In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [11]:
!pip install unsloth

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


使用本地环境变量

In [ ]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HUGGINGFACE_TOKEN")
login(hf_token)

In [4]:
## 使用colab环境变量
from huggingface_hub import login
from google.colab import userdata

hf_token = userdata.get('deepseekr1')
login(hf_token)

可视化训练过程，将数据报送到wandb中

In [ ]:
import wandb

wandb.login(key="your token")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [ ]:
import wandb

# 直接将你的 wandb token 写在这里，不需要调用 user_secrets
wb_token = "your token"

wandb.login(key=wb_token)
run = wandb.init(
    project='Fine-tune-DeepSeek-R1-Distill-Llama-8B on Medical COT Dataset',
    job_type="training",
    anonymous="allow"
)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


## Loading the model and tokenizer

In [8]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_token,
)

==((====))==  Unsloth 2025.2.4: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.9k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

## Model inference before fine-tuning

In [12]:
prompt_style = """以下是一条任务指令，并附带了提供额外上下文的信息。
请根据该请求写出一个合适的回答。
在回答之前，请仔细思考问题，并生成一个逐步的思考链以确保回答的逻辑性和准确性。

### 指令:
你是一位具有高级临床推理、诊断和治疗规划知识的医学专家。
请回答以下医学问题。

### 问题:
{}

### 回答:
<思考>{}
"""

In [13]:

# 中文医学问题
question = "一位61岁的女性，长期存在在咳嗽或打喷嚏等活动中出现不自主尿液丢失，但夜间无漏尿症状，经妇科检查和Q-tip测试。基于这些发现，膀胱测压最有可能显示其残余容量和逼尿肌收缩的情况如何？"

# 启动快速推理（假设 FastLanguageModel 支持中文模型同样适用）
FastLanguageModel.for_inference(model)  # Unsloth 已实现2倍加速推理！

# 根据提示模板和问题构造输入
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# 模型生成答案
outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)

# 解码输出，并根据“### 回答:”标签提取回答内容
response = tokenizer.batch_decode(outputs)
print(response[0].split("### 回答:")[1])


<思考>
首先，我需要明确患者的症状和检查结果。患者是61岁女性，长期存在在咳嗽或打喷嚏等活动中出现不自主的尿液丢失，但夜间没有漏尿。妇科检查和Q-tip测试已经进行了。

接下来，膀胱测压通常用于评估膀胱的残余容量和逼尿肌的收缩情况。在咳嗽或打喷嚏时的不自主排尿，可能与膀胱的功能异常有关，比如膀胱舒张力下降或逼尿肌功能障碍。

考虑到患者在夜间没有漏尿，这可能意味着夜间膀胱的舒张力足够维持尿液的闭锁状态，而在白天，尤其是进行咳嗽或打喷嚏时，膀胱的压力可能不足以维持尿液的闭锁，导致不自主排尿。

膀胱测压的结果可能显示膀胱的残余容量和逼尿肌的收缩情况。如果膀胱测压显示膀胱残余容量较大，可能意味着膀胱的舒张力下降，无法有效闭锁尿道；如果逼尿肌收缩情况不佳，可能进一步影响膀胱的功能。

因此，膀胱测压最有可能显示其残余容量较大，逼尿肌收缩情况不佳，这与患者在咳嗽或打喷嚏时出现不自主排尿的症状相符。
</思考>
回答：膀胱测压可能显示膀胱残余容量较大，逼尿肌收缩情况不佳。
</think>

### 问题回复：

根据患者的症状和检查结果，膀胱测压可能显示膀胱残余容量较大，逼尿肌收缩情况不佳。

### 分步解释：

1. **患者症状分析**：
   - 患者在咳嗽或打喷嚏时出现不自主排尿，但夜间无漏尿。
   - 这可能表明白天膀胱功能异常，夜间膀胱功能正常或足够。

2. **检查结果**：
   - 妇科检查和Q-tip测试已经完成，但未提及具体结果。

3. **膀胱测压的意义**：
   - 膀胱测压用于评估膀胱的残余容量和逼尿肌功能。
   - 残余容量大可能意味着膀胱舒张力下降，无法有效闭锁尿道。
   - 逼尿肌收缩不佳可能导致膀胱功能障碍，尤其在活动中。

4. **结论**：
   - 结合症状和检查，膀胱测压结果可能显示膀胱残余容量较大，逼尿肌收缩情况不佳。<｜end▁of▁sentence｜>


In [14]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)


Unsloth 2025.2.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
model = FastLanguageModel.get_peft_model(
    model=model,  # 待微调的模型
    r=8,  # 将 LoRA 分解的秩从 16 降低到 8，减少额外参数数量
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        # 如果模型不需要对 gate_proj、up_proj 和 down_proj 应用 LoRA，可不传这些模块
    ],
    lora_alpha=32,  # 增大缩放因子以补偿降低的 r，提升低秩适配层的表达能力
    lora_dropout=0.1,  # 添加一定的 dropout 防止过拟合（值可根据实际情况调整）
    bias="none",  # 不修改偏置项
    use_gradient_checkpointing=True,  # 启用梯度检查点，节省显存；如果需要超长上下文，可改为 "unsloth"
    random_state=3407,  # 固定随机种子，确保训练可复现
    use_rslora=True,  # 启用 RSLoRA 变体（根据实际需求设置）
    loftq_config={"scaling": 1.0, "alpha": 1.0},  # 示例配置，可根据需求进一步调整
)

## Loading and processing the dataset

In [16]:
train_prompt_style = """以下是一条任务指令，并附带提供额外上下文信息的输入。
请根据该请求写出一个合适的回答。
在回答之前，请仔细思考问题，并生成一个逐步的思考链以确保回答的逻辑性和准确性。

### 指令:
你是一位具有高级临床推理、诊断和治疗规划知识的医学专家。
请回答以下医学问题。

### 问题:
{}

### 回答:
<思考>
{}
</思考>
{}"""

In [23]:
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN


def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }


In [18]:
# 获取结束符，必须添加 EOS_TOKEN
EOS_TOKEN = tokenizer.eos_token

# 针对 shibing624/medical 数据集（finetune 配置）的字段进行格式化：
def formatting_prompts_func(examples):
    # 从数据集中提取三个字段：instruction, input, output
    instructions = examples["instruction"]
    inputs_ = examples["input"]
    outputs = examples["output"]
    texts = []
    for inst, inp, out in zip(instructions, inputs_, outputs):
        # 如果 input 不为空，则将 instruction 和 input 组合成问题内容
        question = inst + ("\n" + inp if inp.strip() != "" else "")
        # 目前没有提供复杂的逐步思考链，可留空
        cot = ""
        # 将问题、思考链和最终回答填充到中文模板中，并在末尾添加 EOS_TOKEN
        text = train_prompt_style.format(question, cot, out) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

In [24]:
from datasets import load_dataset
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT","en", split = "train[0:500]",trust_remote_code=True)
dataset = dataset.map(formatting_prompts_func, batched = True,)
dataset["text"][0]

README.md:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

medical_o1_sft.json:   0%|          | 0.00/74.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25371 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

"Below is an instruction that describes a task, paired with an input that provides further context.\nWrite a response that appropriately completes the request.\nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nYou are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.\nPlease answer the following medical question.\n\n### Question:\nA 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?\n\n### Response:\n<think>\nOkay, let's think about this step by step. There's a 61-year-old woman here who's been dealing with involuntary urine leakages whenever she's doing something that ups her abdomi

In [27]:
from datasets import load_dataset

# 加载数据集，读取所有样本
dataset = load_dataset("shibing624/medical", "finetune", split="train", trust_remote_code=True)

# 打印数据集的列名，确认有哪些字段
print("数据集的字段：", dataset.column_names)

# 定义格式化函数，生成符合模板要求的 "text" 字段
def formatting_prompts_func(examples):
    new_texts = []
    # 根据数据集实际字段名称调整这里的字段
    # 这里假设数据集中包含 "instruction" 和 "output" 两个字段
    for instruction, output in zip(examples["instruction"], examples["output"]):
        formatted_text = f"问题: {instruction}\n回答: {output}"
        new_texts.append(formatted_text)
    return {"text": new_texts}

# 对数据集应用格式化函数，生成符合模板要求的文本（即 "text" 字段）
dataset = dataset.map(formatting_prompts_func, batched=True)

# 查看第一个生成的文本
print(dataset["text"][0])

数据集的字段： ['instruction', 'input', 'output']


Map:   0%|          | 0/2066589 [00:00<?, ? examples/s]

问题: 血热的临床表现是什么?
回答: 初发或复发病不久。皮疹发展迅速，呈点滴状、钱币状或混合状。常见丘疹、斑丘疹、大小不等的斑片，潮红、鲜红或深红色。散布于体表各处或几处，以躯干、四肢多见，亦可先从头面开始，逐渐发展至全身。新皮疹不断出现，表面覆有银白色鳞屑，干燥易脱落，剥刮后有点状出血。可有同形反应;伴瘙痒、心烦口渴。大便秘结、小便短黄，舌质红赤，苔薄黄或根部黄厚，脉弦滑或滑数。血热炽盛病机，主要表现在如下四个面：一、热象：血热多属阳盛则热之实性、热性病机和病证、并表现出热象。二、血行加速：血得热则行，可使血流加速，且使脉道扩张，络脉充血，故可见面红目赤，舌色深红（即舌绛）等症。三、动血：在血行加速与脉道扩张的基础上，血分有热，可灼伤脉络，引起出血，称为“热迫血妄行”，或称动血。四、扰乱心神：血热炽盛则扰动心神，心主血脉而藏神，血脉与心相通，故血热则使心神不安，而见心烦，或躁扰发狂等症。


In [ ]:
from datasets import load_dataset

dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "en",split = "train[0:500]")
dataset = dataset.map(formatting_prompts_func, batched = True,)
dataset["text"][0]

## Setting up the model

In [57]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=8,    # 增加每设备批大小，提高梯度估计稳定性（如果显存允许）
        gradient_accumulation_steps=2,      # 保持较低的累积步数，有效批大小 = 8 × 2 = 16
        warmup_steps=1000,                  # 预热步数增至 1000，保证学习率平滑上升，适应较长的训练过程
        max_steps=8000,                    # 总训练步数设为 10000
        learning_rate=2e-5,                 # 进一步降低学习率，确保参数更新更细致
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=100,                  # 日志打印间隔设置为 100 步，减少过于频繁的输出
        optim="adamw_8bit",                 # 采用 adamw_8bit 优化器
        weight_decay=0.01,                  # 保持 0.01 的权重衰减
        lr_scheduler_type="linear",         # 使用线性学习率衰减策略
        seed=3407,
        output_dir="outputs",
    ),
)

## Model training

In [58]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,066,589 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 2
\        /    Total batch size = 16 | Total steps = 8,000
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
100,2.048200
200,2.069200
300,2.042000
400,2.025700
500,2.010200
600,2.003700
700,1.999700
800,2.002000
900,2.022900
1000,2.013200


Step,Training Loss
100,2.048200
200,2.069200
300,2.042000
400,2.025700
500,2.010200
600,2.003700
700,1.999700
800,2.002000
900,2.022900
1000,2.013200


In [59]:
import wandb

# 其他 wandb 相关代码
wandb.finish()

train/epoch,▃▇█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/global_step,▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▁▂▂▂▃▃▃▄▄▅▅▅▆▆▇▇▇▇█
train/grad_norm,▅▂▅▂▁▄▃▄▆▇█▇▄▂▆▄▄▅▆▃▄▅▃▃▃▃▄▃▆▄▅▅▄▃▄▄▃▅▃▅
train/learning_rate,█▄▃▁▂▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,█▄▆▇▆▅▆▅▅▅▄▄▃▃▃▃▃▃▃▃▃▂▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁
total_flos,3.4129774623422546e+18
train/epoch,0.06194
train/global_step,8000
train/grad_norm,1.1344
train/learning_rate,0
train/loss,2.0147


## Model inference after fine-tuning

In [29]:
question = "A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?"


FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])



<think>
Okay, so let's think about this. We have a 61-year-old woman who's been dealing with involuntary urine loss during things like coughing or sneezing, but she's not leaking at night. That suggests she might have some kind of bladder control issue, but it's not the classic overactive bladder situation. 

When they did the Q-tip test, they found that the bladder wasn't fully emptying. Hmm, that makes me think there's some residual volume left in her bladder after she's done urinating. That residual volume can be a big problem because it's not getting emptied, which can lead to more frequent urination or even some of those involuntary losses we're talking about.

Now, when they do a cystometry, which is like a test to check how the bladder behaves, we expect to see some things. If there's residual volume, the bladder won't fully empty, which could mean it's not contracting properly. That would affect how the bladder behaves during activities.

Given all this, it makes sense that th

In [67]:
# 定义中文医学问题
question = "一位61岁的女性长期存在咳嗽或打喷嚏时出现不自主的尿液外漏，但夜间无漏尿。她接受了妇科检查和Q-tip测试。基于这些检查结果，膀胱测压最有可能显示其残余尿量和逼尿肌收缩的情况如何？"

# 使用快速推理模式（Unsloth 加速）
FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!

# 利用预定义的中文模板（prompt_style）构造输入，第二个占位符为空（这里可用于填充思考链）
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# 生成模型输出
outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)

# 解码输出，并通过“### 回答:”分隔符提取模型回答部分
response = tokenizer.batch_decode(outputs)
print(response[0].split("### 回答:")[1])


<思考>
根据您提供的信息，患者的症状是夜间无漏尿，白天有不自主的尿液外漏。Q-Tip测试结果显示膀胱残余尿量为50毫升，膀胱测压为20厘米水压。膀胱测压为20厘米水压，膀胱残余尿量为50毫升，膀胱容量为100毫升。膀胱测压为20厘米水压，膀胱容量为100毫升，膀胱残余尿量为50毫升。膀胱测压为20厘米水压，膀胱容量为100毫升，膀胱残余尿量为50毫升。膀胱测压为20厘米水压，膀胱容量为100毫升，膀胱残余尿量为50毫升。膀胱测压为20厘米水压，膀胱容量为100毫升，膀胱残余尿量为50毫升。膀胱测压为20厘米水压，膀胱容量为100毫升，膀胱残余尿量为50毫升。膀胱测压为20厘米水压，膀胱容量为100毫升，膀胱残余尿量为50毫升。膀胱测压为20厘米水压，膀胱容量为100毫升，膀胱残余尿量为50毫升。膀胱测压为20厘米水压，膀胱容量为100毫升，膀胱残余尿量为50毫升。膀胱测压为20厘米水压，膀胱容量为100毫升，膀胱残余尿量为50毫升。膀胱测压为20厘米水压，膀胱容量为100毫升，膀胱残余尿量为50毫升。膀胱测压为20厘米水压，膀胱容量为100毫升，膀胱残余尿量为50毫升。膀胱测压为20厘米水压，膀胱容量为100毫升，膀胱残余尿量为50毫升。膀胱测压为20厘米水压，膀胱容量为100毫升，膀胱残余尿量为50毫升。膀胱测压为20厘米水压，膀胱容量为100毫升，膀胱残余尿量为50毫升。膀胱测压为20厘米水压，膀胱容量为100毫升，膀胱残余尿量为50毫升。膀胱测压为20厘米水压，膀胱容量为100毫升，膀胱残余尿量为50毫升。膀胱测压为20厘米水压，膀胱容量为100毫升，膀胱残余尿量为50毫升。膀胱测压为20厘米水压，膀胱容量为100毫升，膀胱残余尿量为50毫升。膀胱测压为20厘米水压，膀胱容量为100毫升，膀胱残余尿量为50毫升。膀胱测压为20厘米水压，膀胱容量为100毫升，膀胱残余尿量为50毫升。膀胱测压为20厘米水压，膀胱容量为100毫升，膀胱残余尿量为50毫升。膀胱测压为20厘米水压，膀胱容量为100毫升，膀胱残余尿量为50毫升。膀胱测压为20厘米水压，膀胱容量为100毫升，膀胱残余尿量为50毫升。膀胱测压为20厘米水压，膀胱容量为100毫升，�


In [ ]:
# 定义中文医学问题
question = "怀孕后嘴巴很淡怎么办？"

# 构造新的 prompt，明确要求模型只给出最终答案，不展示中间思考过程，
# 并确保回答简洁、精炼且不重复
prompt = (
    f"问题：{question}\n"
    "请直接给出最终答案，不需要显示任何中间思考过程，并确保回答简洁、精炼且不重复。\n"
    "### 回答:"
)

# 利用新的 prompt 构造输入文本
inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

# 使用模型生成回答，同时设置 no_repeat_ngram_size 和 repetition_penalty 参数防止重复
outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,           # 可根据需要调整生成 token 数量
    no_repeat_ngram_size=3,       # 防止连续3个 token 重复
    repetition_penalty=1.2,       # 对重复生成的内容进行惩罚
    use_cache=True,
)

# 解码输出，并通过“### 回答:”分隔符提取最终答案部分
response = tokenizer.batch_decode(outputs)
final_answer = response[0].split("### 回答:")[-1].strip()

print(final_answer)

In [79]:
# 定义中文医学问题
question = "感冒了怎么办"

# 构造新的 prompt，明确要求模型只给出最终答案，不展示中间思考过程，
# 并确保回答简洁、精炼且不重复
prompt = (
    f"问题：{question}\n"
    "请直接给出最终答案，不需要显示任何中间思考过程，并确保回答简洁、精炼且不重复。\n"
    "### 回答:"
)

# 利用新的 prompt 构造输入文本
inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

# 使用模型生成回答，同时设置 no_repeat_ngram_size 和 repetition_penalty 参数防止重复
outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,           # 可根据需要调整生成 token 数量
    no_repeat_ngram_size=3,       # 防止连续3个 token 重复
    repetition_penalty=1.2,       # 对重复生成的内容进行惩罚
    use_cache=True,
)

# 解码输出，并通过“### 回答:”分隔符提取最终答案部分
response = tokenizer.batch_decode(outputs)
final_answer = response[0].split("### 回答:")[-1].strip()

print(final_answer)

1.如果是急性呼吸道传染病，应注意休息和多喝水。不要吃太热或刺激性的食物，以免加剧症状。如果有咳嗽，您可以在平时适当用一些喷雾剂来缓解痰液的排泄。但你要小心，如果你的喉部很疼，你不能使用药品，而必须去医院诊断并进行治疗。在这种情况下，一般会服用止痛类药，如阿司匹林（可用于成人），氨酚黄那敏片（对儿童有效）。对于严重的肺炎患者，可以考虑注射抗生素，因为它更能消除细菌侵袭所造成的疾病。

2.在家里生活期间，尽量避免接触冷空气以及寒风等，有助于预防鼻窦炎。由于我们经常处于室内环境中，所以容易滋长各种微生物，因此应该定期清洗房间，保持卫生条件以减少污染机会。此外，在家庭内部，我们通常会采用一种高效率电饭锅，它将煮沸我们的饮料，使其蒸发，从而产生更多的湿度，这样就不会让人流汗过多，同时还能使人们感到舒畅。不仅如此，当冬天到来后，我们也可能会出现许多其他健康相关的问题，比如说干燥，或者就是非常疲惫无力，这种时候，大家都应该更加关注自己的身体状况，也希望自己能够早日恢复健康。

3.您好，孩子被感冒的情况一般都是先自觉头晕乏力，然后再次出现恶心呕吐，伴随着低烧甚至还有轻微的全身酸痛，建议带他及时检查血象，看看是否存在白细胞增高现象，如果没有的话，就说明不是因为细菌引起的感冒，只是因受凉导致的上火，要留意观察他的精神状态如何，再决定采取什么样的措施实施救治。当然也有可能通过物理方法促进散热，比如做推拿法或者按摩疗法，对此也是有一定的效果的，但一定要配合医生的意见实施相应的手术操作。以上这些内容，我相信我已经告诉你该怎样处理这个问题。你现在应当坚持下去，现在只是暂停了一段时间，还没完全康复。我想这位朋友肯定会明白我的意思吧？所以，请继续坚持住！祝福每个人拥有一个快乐美好的夜晚！

综上所述,感冒是由很多原因引来的,主要表现为患儿的体温升高,同时又感觉头昏,四肢抽筋,一旦发现自身皮肤颜色变深,口唇稍微泛紫,则表明生命危险极大,那么就在紧急情况下,千万不可拖延时间,立即送往附近三甲级别的正规医院。除了积极地补充营养之外,为了保证感冒者的新鲜果蔬供应,比如苹果,橙子,柠檬,葡萄等,这几个食品具有降糖,提高机体抵抗力的功效,特别是在季节变化的时候,它们具备保护人的能力,尤其是那些易发生感冒的人群,只要合理搭配摄入足够数量的维生素C,就会帮助他们快速克服感冒困难。因此在准备面临今年春季的一场

## Saving the model locally

In [31]:
new_model_online = "kingabzpro/DeepSeek-R1-Medical-COT"
new_model_local = "DeepSeek-R1-Medical-COT"
model.save_pretrained(new_model_local) # Local saving
tokenizer.save_pretrained(new_model_local)


('DeepSeek-R1-Medical-COT/tokenizer_config.json',
 'DeepSeek-R1-Medical-COT/special_tokens_map.json',
 'DeepSeek-R1-Medical-COT/tokenizer.json')

In [80]:
new_model_online = "beita6969/deepseek-r1-medical-response"
new_model_local = "deepseek-r1-medical-response"


model.save_pretrained(new_model_local) # Local saving
tokenizer.save_pretrained(new_model_local)

('deepseek-r1-medical-response/tokenizer_config.json',
 'deepseek-r1-medical-response/special_tokens_map.json',
 'deepseek-r1-medical-response/tokenizer.json')

## Pushing the model to Hugging Face hub

In [81]:
model.push_to_hub(new_model_online) # Online saving
tokenizer.push_to_hub(new_model_online) # Online saving

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Saved model to https://huggingface.co/beita6969/deepseek-r1-medical-response


README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [82]:
model.save_pretrained_merged(new_model_local, tokenizer, save_method = "merged_16bit",)
model.push_to_hub_merged(new_model_online, tokenizer, save_method = "merged_16bit")

Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 6.0G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.5 out of 83.48 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [00:00<00:00, 45.65it/s]


Unsloth: Saving tokenizer... Done.
Done.


Unsloth: You are pushing to hub, but you passed your HF username = beita6969.
We shall truncate beita6969/deepseek-r1-medical-response to deepseek-r1-medical-response


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/beita6969/deepseek-r1-medical-response
